# Введение в анализ данных (первый семестр)
## Домашнее задание №4 - Участие в конкурсе Quora Question Pairs

<b>Что вам нужно будет сделать</b><br>
До <b>23:59 7 мая</b> сделать хотя бы один submission в соревновании, а в промежуток <b>с 9 по 10 мая</b> выложить отчет (код в jupyter notebook, который генерирует ваше решение) в топик на kaggle, который я чуть позже создам (правила kaggle запрещают обмениваться кодом за пределами платформы).
Баллы за это задание будут начисляться следующим образом (только при наличии отчета):<br>
результат < 0.5 - 6 баллов<br>
результат < 0.4 - 8 баллов<br>
результат < 0.35 - 10 баллов<br>
остальные критерии будут определены постфактум (при оценивании будут учитываться ваше место в лидерборде и отчет)

<h3>Подготовка</h3>

Импорт всех необходимых пакетов:

In [11]:
import gc
import time
from datetime import datetime
import cPickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
%matplotlib inline

<h3>Загрузка и предобработка тренировочных данных</h3>

In [2]:
train_df = pd.read_csv('data/train.csv')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404290 non-null object
question2       404288 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [3]:
train_df[train_df['question2'].isnull()]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0


In [4]:
train_df = train_df.dropna().reset_index(drop=True)
train_df.head(n=10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404288 entries, 0 to 404287
Data columns (total 6 columns):
id              404288 non-null int64
qid1            404288 non-null int64
qid2            404288 non-null int64
question1       404288 non-null object
question2       404288 non-null object
is_duplicate    404288 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


<h3>Извлечение признаков</h3>

In [6]:
BOW = CountVectorizer(max_df=0.999, min_df=50, max_features=10000, analyzer='char', 
                      ngram_range=(1,10), binary=True, lowercase=True, stop_words=None)

unique_questions = pd.concat((train_df['question1'],train_df['question2'])).unique()

%time BOW.fit(unique_questions)

Wall time: 1min 25s


CountVectorizer(analyzer='char', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.999, max_features=None, min_df=1000,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [7]:
%time BOW_q1, BOW_q2 = BOW.transform(train_df.question1), BOW.transform(train_df.question2)

Wall time: 2min 18s


In [8]:
labels = train_df.is_duplicate

Введём следующий признак:<br>
0, если последовательность букв находится как в BOW первых вопросов, так и в BOW вторых вопросов, или отсутствует и там, и там<br>
-1, если последовательность букв находится только в одном BOW, и отсутствует в другом<br>

In [9]:
X = -(BOW_q1 != BOW_q2).astype(int)
y = labels.values

<h3>Обучение модели на тренировочных данных</h3>

In [12]:
%time model_lr = LogisticRegression(C=0.1, solver='sag', class_weight={1: 0.46, 0: 1.32}).fit(X, y)

Wall time: 1min 4s


<h3>Загрука и предобработка тестовых данных</h3>

In [13]:
test_df = pd.read_csv('data/test.csv')
test_df.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345796 entries, 0 to 2345795
Data columns (total 3 columns):
test_id      int64
question1    object
question2    object
dtypes: int64(1), object(2)
memory usage: 53.7+ MB


In [18]:
test_df.loc[test_df['question1'].isnull(), ['question1','question2']] = ''
test_df.loc[test_df['question2'].isnull(), ['question1','question2']] = ''

<h3>Извлечение признаков</h3>

In [19]:
%time BOW_q1_test = BOW.transform(test_df.question1)
%time BOW_q2_test = BOW.transform(test_df.question2)

,question1,question2


Так как векторизация тестовых вопросов занимает очень много времени, сериализуем полученные множества:

In [ ]:
with open('BOW_q1_test.pkl', 'wb') as fid:
    cPickle.dump(BOW_q1_test, fid)
with open('BOW_q2_test.pkl', 'wb') as fid:
    cPickle.dump(BOW_q2_test, fid)

Десериализуем ранее полученные BOW для тестовых вопросов:

In [ ]:
with open('BOW_q1_test.pkl', 'r') as fid:
    BOW_q1_test_loaded = cPickle.load(fid)
with open('BOW_q2_test.pkl', 'r') as fid:
    BOW_q2_test_loaded = cPickle.load(fid)

In [ ]:
X_test = -(BOW_q1_test_loaded != BOW_q2_test_loaded).astype(int)
X_test.shape

<h3>Применение модели на тестовых данных</h3>

In [ ]:
predicted = model.predict_proba(X_test)

<h3>Сохранение предсказаний</h3>

In [ ]:
submission = pd.concat([test_df.test_id.astype(int), pd.DataFrame(test_predicted[:,1], 
                                        columns=['is_duplicate'], index=test_df.index)], axis=1)
submission.head()

In [ ]:
suffix = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
sumbission.to_csv('./answers/sumbission_%s.csv' % suffix, index=False)

In [27]:
!jupyter --version

4.2.0
